In [ ]:
import os

from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from src.data_handler import DataHandler
from src.analysis_logic import FitMethods, AnalysisLogic
from src.helper_functions import format_exponent_as_str

plt.style.use('seaborn')
plt.rcParams['image.cmap'] = sns.color_palette("mako", as_cmap=True)

In [ ]:
tip_2R16 = DataHandler(measurement_folder="20221206_FR0612-F2-2R16_PCB_UHVstage_rotated")
analysis = AnalysisLogic()

In [ ]:
saturation = tip_2R16.load_measurements_into_dataclass_list(measurement_str="Saturation")

# Using old scheme for Saturation measurement

# ND filter position 1

In [ ]:
fig, ax = plt.subplots()
sat = saturation[0]

conv = 1.6e3
x, y = sat.data["Power (W)"] * conv, sat.data["Avg. counts"] / 1e3
ax.plot(x, y, "o-")

fit_x, fit_y, res = sat.analysis.perform_fit(x, y, fit_function=FitMethods.decayexponential)

ax.plot(fit_x, fit_y)

ax.set_xlabel("Power at objective (uW)")
ax.set_ylabel("Counts (kcts/s)")

x_pre = np.arange(sat.data["Power (W)"][0] * conv, sat.data["Power (W)"][len(sat.data["Power (W)"]) - 1] * 5 * conv)
y_pre = res.eval(x=x_pre)

ax.plot(x_pre, y_pre)

# ND filter position 2

In [ ]:
laser_powermeter_factor = 75e-6 / 100e-3 
powermeter_objective_factor = 4

# in uW
conversion_factor = laser_powermeter_factor * powermeter_objective_factor * 1e6

In [ ]:
fig, ax = plt.subplots()

sat = saturation[1]

x, y = sat.data["Power (W)"] * conversion_factor, sat.data["Avg. counts"] / 1e3
ax.plot(x, y, "o-")

fit_x, fit_y, res = sat.analysis.perform_fit(x, y, fit_function=FitMethods.decayexponential)

ax.plot(fit_x, fit_y)

ax.set_xlabel("Power at objective (uW)")
ax.set_ylabel("Counts (kcts/s)")

x_pre = np.arange(sat.data["Power (W)"][0] * conversion_factor, sat.data["Power (W)"][len(sat.data["Power (W)"]) - 1] * conversion_factor * 3)
y_pre = res.eval(x=x_pre)
ax.plot(x_pre, y_pre)

# ND filter position 3

In [ ]:
laser_powermeter_factor = 100e-6 / 100e-3 
powermeter_objective_factor = 4

# in uW
conversion_factor = laser_powermeter_factor * powermeter_objective_factor * 1e6

In [ ]:
fig, ax = plt.subplots()

sat = saturation[2]

x, y = sat.data["Power (W)"] * conversion_factor, sat.data["Avg. counts"] / 1e3
ax.plot(x, y, "o-")

fit_x, fit_y, res = sat.analysis.perform_fit(x, y, fit_function=FitMethods.decayexponential)
#ax.plot(fit_x, fit_y)
ax.set_xlabel("Power at objective (uW)")
ax.set_ylabel("Counts (kcts/s)")

x_pre = np.arange(sat.data["Power (W)"][0] * conversion_factor, sat.data["Power (W)"][len(sat.data["Power (W)"]) - 1] * conversion_factor * 2)
y_pre = res.eval(x=x_pre)
ax.plot(x_pre, y_pre)

sat = saturation[3]

x, y = sat.data["Power (W)"] * conversion_factor, sat.data["Avg. counts"] / 1e3
ax.plot(x, y, "o-")

fit_x, fit_y, res = sat.analysis.perform_fit(x, y, fit_function=FitMethods.decayexponential)
#ax.plot(fit_x, fit_y)
ax.set_xlabel("Power at objective (uW)")
ax.set_ylabel("Counts (kcts/s)")

x_pre = np.arange(sat.data["Power (W)"][0] * conversion_factor, sat.data["Power (W)"][len(sat.data["Power (W)"]) - 1] * conversion_factor * 2)
y_pre = res.eval(x=x_pre)
ax.plot(x_pre, y_pre)

# Autocorrelation

In [ ]:
autocorr = tip_2R16.load_measurements_into_dataclass_list(measurement_str="Autocorrelation")

In [ ]:
auto = autocorr[-1]

fig, ax = plt.subplots()

x, y = auto.data["Time (ps)"] / 1e3, auto.data["g2(t) norm"]
ax.plot(x, y)

fit_x, fit_y, result = analysis.perform_fit(x, y, fit_function=FitMethods.antibunching, estimator="dip")

ax.plot(fit_x, fit_y)
ax.set_title(f"{auto.filename}, g2(0) = {result.eval(x=0):.2f}")

ax.axhspan(0.3, 0.5, alpha=0.5, color="C1")

ax.set_ylabel("g(2)")
ax.set_xlabel("Time (ns)")

# Using new scheme for Saturation measurement

In [ ]:
sat = saturation[-1]
sat.data["UHV objective power (W)"] *= 1e6

In [ ]:
fig, ax = plt.subplots()

x, y = sat.data["UHV objective power (W)"], sat.data["Avg. counts"] / 1e3
yerr = np.array(sat.data["Upper count errors"] / 1e3, sat.data["Lower count errors"] / 1e3)
ax.errorbar(x, y, fmt="o", yerr=yerr)

fit_x, fit_y, res = sat.analysis.perform_fit(x, y, fit_function=FitMethods.hyperbolicsaturation)
ax.plot(fit_x, fit_y)

ax.set_xlabel("Power at objective (μW)")

#x_pre = np.linspace(sat.data["UHV objective power (W)"][0], sat.data["UHV objective power (W)"][len(sat.data["UHV objective power (W)"]) - 1] * 2, 50)
#y_pre = res.eval(x=x_pre)
#ax.plot(x_pre, y_pre)